In [2]:
import numpy as np 




In [ ]:
K = np.zeros((3,3,3,3))


transverse_modulus = 2432.29e6 #Pa
principal_modulus = 2367.10e6 # Pa

transvere_poisson = 0.24
principal_poisson = 0.34

def transverse_isotropy(E1, E2, nu1, nu2):
    #E1->elasticity modulus in the isotropic plane
    #E2->e.m. in transverse axis
    C11 = E1*(1-nu2**2)/(1-nu1**2)
    C12 = E1*nu1/(1-nu1**2)
    C13 = E2*nu2/(1-nu2**2)
    C33 = E2/(1-nu2**2)
    C44 = E1/(2*(1+nu1))
    C66 = 0.5*(C11-C12)
    return C11, C12, C13, C33 , C44, C66

C11, C12, C13, C33 , C44, C66 = transverse_isotropy(transverse_modulus, principal_modulus, transvere_poisson, principal_poisson)

def Stiffnesstensor(C11, C12, C13, C33, C44, C66):
    C = np.zeros((3,3,3,3))
    C[0, 0, 0, 0] = C11
    C[1, 1, 1, 1] = C11
    C[0, 0, 1, 1] = C12
    C[1, 1, 0, 0] = C12
    C[0, 0, 2, 2] = C13
    C[2, 2, 0, 0] = C13
    C[1, 1, 2, 2] = C13
    C[2, 2, 1, 1] = C13
    C[2, 2, 2, 2] = C33
    C[1, 2, 1, 2] = C44
    C[2, 1, 2, 1] = C44
    C[2, 1, 1, 2] = C44
    C[1, 2, 2, 1] = C44
    C[0, 2, 0, 2] = C44
    C[2, 0, 2, 0] = C44
    C[2, 0, 0, 2] = C44
    C[0, 2, 2, 0] = C44
    C[0, 1, 0, 1] = C66
    C[1, 0, 1, 0] = C66
    C[1, 0, 0, 1] = C66
    C[0, 1, 1, 0] = C66
    return C 


C = Stiffnesstensor(C11, C12, C13, C33 , C44, C66 )


In [2]:
from skfem import *
from skfem.helpers import *

# assuming C is defined as above ...



In [23]:
## HARDCODED FORCE IMPLECATION FROM ONE DIRECTION ##
import pytetwild
import meshio
import numpy as np

import numpy as np
from skfem import *
from skfem.models.elasticity import (linear_elasticity, lame_parameters, linear_stress)
from skfem.helpers import dot, sym_grad
from skfem.visuals.matplotlib import draw
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

transverse_modulus = 2432.29e6 #Pa
principal_modulus = 2367.10e6 # Pa

transvere_poisson = 0.24
principal_poisson = 0.34

def transverse_isotropy(E1, E2, nu1, nu2):
    #E1->elasticity modulus in the isotropic plane
    #E2->e.m. in transverse axis
    C11 = E1*(1-nu2**2)/(1-nu1**2)
    C12 = E1*nu1/(1-nu1**2)
    C13 = E2*nu2/(1-nu2**2)
    C33 = E2/(1-nu2**2)
    C44 = E1/(2*(1+nu1))
    C66 = 0.5*(C11-C12)
    return C11, C12, C13, C33 , C44, C66

C11, C12, C13, C33 , C44, C66 = transverse_isotropy(transverse_modulus, principal_modulus, transvere_poisson, principal_poisson)

def Stiffnesstensor(C11, C12, C13, C33, C44, C66):
    C = np.zeros((3,3,3,3))
    C[0, 0, 0, 0] = C11
    C[1, 1, 1, 1] = C11
    C[0, 0, 1, 1] = C12
    C[1, 1, 0, 0] = C12
    C[0, 0, 2, 2] = C13
    C[2, 2, 0, 0] = C13
    C[1, 1, 2, 2] = C13
    C[2, 2, 1, 1] = C13
    C[2, 2, 2, 2] = C33
    C[1, 2, 1, 2] = C44
    C[2, 1, 2, 1] = C44
    C[2, 1, 1, 2] = C44
    C[1, 2, 2, 1] = C44
    C[0, 2, 0, 2] = C44
    C[2, 0, 2, 0] = C44
    C[2, 0, 0, 2] = C44
    C[0, 2, 2, 0] = C44
    C[0, 1, 0, 1] = C66
    C[1, 0, 1, 0] = C66
    C[1, 0, 0, 1] = C66
    C[0, 1, 1, 0] = C66
    return C 


C = Stiffnesstensor(C11, C12, C13, C33 , C44, C66 )

m = MeshTet()

input_file = "egg.stl"


# # Initialisieren des Netzes und der Elemente

stl : meshio.Mesh = meshio.read(f"C:/Users/AlexP/Desktop/BA/STL_Files/{input_file}")
vertices, tetrahedras = pytetwild.tetrahedralize(stl.points, stl.cells_dict["triangle"])

cells = meshio.CellBlock('tetra', tetrahedras)
    # write intermediate file
#meshio.write_points_cells(f"C:/Users/AlexP/Desktop/BA/Meshes/{output_file}", vertices, [cells])

    # in case you want to directly continue with it
p = np.array([vertices[:,0],  vertices[:,1],  vertices[:,2]], dtype=np.float64)
t = np.array(tetrahedras, dtype=np.float64).T
m = MeshTet(p, t)






#m = m.load("C:/Users/AlexP/Desktop/BA/Meshes/egg.msh")

e1 = ElementTetP1() # ElementHex1() # ElementTetP1()
e = ElementVector(e1)
ib = Basis(m, e, MappingIsoparametric(m, e1), 3)


@BilinearForm
def anisotropic(u, v, _):
    epsu = 0.5 * (grad(u) + transpose(grad(u)))
    epsv = 0.5 * (grad(v) + transpose(grad(v)))
    return np.einsum('ijkl,ij...,kl...', C, epsu, epsv)


# assemblierung stiffness matrix
K = asm(anisotropic, ib)

# label dofs
dofs = {
        'left': ib.get_dofs(lambda x: np.isclose(x[0], x[0].min())),
        'right': ib.get_dofs(lambda x: np.isclose(x[0], x[0].max())),
        'up': ib.get_dofs(lambda x: np.isclose(x[1], x[1].max())),
        'down': ib.get_dofs(lambda x: np.isclose(x[1], x[1].min())),
        'front': ib.get_dofs(lambda x: np.isclose(x[2], x[2].max())),
        'back': ib.get_dofs(lambda x: np.isclose(x[2], x[2].min()))
    }

u = ib.zeros()

# force applied to knots
Kraft = e6  
F = np.zeros(u.shape)
up_dofs = dofs['front'].nodal['u^2']
F[up_dofs] = Kraft

# Randbedingungen fixieren
fixed_dofs = np.hstack([dofs['back'].all()])
u[fixed_dofs] = 0

#set dofs and solve DGL
free_dofs = np.setdiff1d(np.arange(K.shape[0]), fixed_dofs)
K_free = K[free_dofs][:, free_dofs]
F_free = F[free_dofs]
u_free = solve(K_free, F_free)
u[free_dofs] = u_free

# calculating upper shift only for the hard coded example
verschiebung_oben = u[up_dofs]


# implement shifts
u[up_dofs] = verschiebung_oben


I = ib.complement_dofs(np.concatenate((dofs['front'], dofs['back'])))
u = solve(*condense(K, x=u, I=I))


factor = 1
m_defo = m.translated(factor * u[ib.nodal_dofs])
# draw(m_defo)
m_defo.save("C:/Users/AlexP/Desktop/BA/Meshes/anisotests_" + '.vtk', {"affected": u[ib.nodal_dofs][0]})

print(dofs)


{'left': <skfem DofsView(MeshTet1, ElementVector) object>
  Number of nodal DOFs: 552 ['u^1' 'u^2' 'u^3'], 'right': <skfem DofsView(MeshTet1, ElementVector) object>
  Number of nodal DOFs: 597 ['u^1' 'u^2' 'u^3'], 'up': <skfem DofsView(MeshTet1, ElementVector) object>
  Number of nodal DOFs: 573 ['u^1' 'u^2' 'u^3'], 'down': <skfem DofsView(MeshTet1, ElementVector) object>
  Number of nodal DOFs: 537 ['u^1' 'u^2' 'u^3'], 'front': <skfem DofsView(MeshTet1, ElementVector) object>
  Number of nodal DOFs: 546 ['u^1' 'u^2' 'u^3'], 'back': <skfem DofsView(MeshTet1, ElementVector) object>
  Number of nodal DOFs: 543 ['u^1' 'u^2' 'u^3']}
[   0    1    2 ... 6627 6628 6629]
[[ 0.          0.          0.         ...  0.         -0.00253755
   0.00394911]
 [ 0.          0.          0.         ...  0.          0.10493831
   0.06540491]
 [ 0.          0.          0.         ...  0.         -0.00032603
   0.00412466]]
